In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from IPython.display import display

In [2]:
data = yf.download(['EURMXN=X','MXN=X'], start='2020-01-01', end='2025-01-24')['Adj Close']
positions = [2400000, 1500000]
conf = 95
long = 1

[*********************100%***********************]  2 of 2 completed


In [3]:
def get_var_cvar(data, positions, conf, long):
    port = data * positions
    port['total'] = port.sum(axis=1)
    port['rendimiento'] = port['total'].pct_change()

    if long == 1:

        var_porcentual = np.percentile(port['rendimiento'].dropna(), 100-conf)
        var_cash = port['total'].iloc[-1] * var_porcentual

        cvar_porcentual = port.query("rendimiento < @var_porcentual")['rendimiento'].mean()
        cvar_cash = port['total'].iloc[-1] * cvar_porcentual

    else:

        var_porcentual = np.percentile(port['rendimiento'].dropna(), conf)
        var_cash = port['total'].iloc[-1] * var_porcentual

        cvar_porcentual = port.query("rendimiento > @var_porcentual")['rendimiento'].mean()
        cvar_cash = port['total'].iloc[-1] * cvar_porcentual

    var_df = pd.DataFrame({
        'VaR_porcentual': [var_porcentual * 100],
        'VaR_cash': [var_cash]
    })

    cvar_df = pd.DataFrame({
        'cVaR_porcentual': [cvar_porcentual * 100],
        'cVaR_cash': [cvar_cash]
    })

    return display(var_df), display(cvar_df)

In [4]:
get_var_cvar(data, positions, conf, long)

,VaR_porcentual,VaR_cash
0,-1.066285,-873595.70265


,cVaR_porcentual,cVaR_cash
0,-1.530191,-1.253669e+06


(None, None)